In [19]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2

import esim_torch
print("导入esim_torch")

import sys
sys.path.append('/home/gwp/rpg_vid2e/upsampling')  # 替换为实际的路径，这个路径包含了upsampling目录
from utils import Upsampler
print("导入Upsampler")


导入esim_torch
导入Upsampler


In [28]:
device = "cuda:0"
upsampler = Upsampler(input_dir="./example/original/seq0", output_dir="./example/upsampled/seq0")
upsampler.upsample()

# 读取原本图片
image_files = sorted(glob.glob("./example/original/seq0/imgs/*.png"))
images = np.stack([cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files])

# 读取上采样后的图片
image_files = sorted(glob.glob("./example/upsampled/seq0/imgs/*.png"))
usampled_images = np.stack([cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files])

print("原本图片的shape：", images.shape, "上采样后图片的shape：", usampled_images.shape)


Processing sequence number ./example/original/seq0


ImageSequence: 100%|██████████| 8/8 [00:11<00:00,  1.45s/it]

原本图片的shape： (9, 260, 346) 上采样后图片的shape： (65, 256, 320)


In [29]:
device = "cuda:0"
upsampler = Upsampler(input_dir="./esim_py/tests/data/images", output_dir="./esim_py/tests/data/upsampled/images") #注意，图像序列要放在文件夹imgs下，且需要有fps.txt记录帧率
upsampler.upsample()

print("Loading images")
# 读取原本图片
image_files = sorted(glob.glob("./esim_py/tests/data/images/images/*.png"))
images = np.stack([cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files])

# 读取上采样后的图片
image_files = sorted(glob.glob("./esim_py/tests/data/upsampled/images/imgs/*.png"))
usampled_images = np.stack([cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files])

print("原本图片的shape：", images.shape, "上采样后图片的shape：", usampled_images.shape)

Processing sequence number ./esim_py/tests/data/images


ImageSequence: 100%|██████████| 160/160 [00:26<00:00,  6.15it/s]


Loading images
原本图片的shape： (161, 180, 240) 上采样后图片的shape： (321, 160, 224)


Loading images
原本图片的shape： (161, 180, 240) 上采样后图片的shape： (321, 160, 224)
